In [55]:
import selenium
from selenium import webdriver
import pandas as pd

In [59]:
browser=webdriver.Chrome('chromedriver.exe')
browser.maximize_window()

In [4]:
browser.get('https://timesofindia.indiatimes.com/archive/year-2018,month-9.cms')

In [19]:
########## collecting all dates ##################33
div=browser.find_element_by_id("calenderdiv")
dates=[]
for i in div.find_elements_by_xpath(".//table/tbody/tr"):
    for k in i.find_elements_by_xpath(".//td/a"):
        date_exist=False
        date=k.get_attribute('href')
        ### searching if date already present in dates list
        for x in dates:
            if(x==date):date_exist=True
        ## adding date if date not present in list
        if date_exist == False:dates.append(date)

In [16]:
dates

['https://timesofindia.indiatimes.com/2018/9/1/archivelist/year-2018,month-9,starttime-43344.cms',
 'https://timesofindia.indiatimes.com/2018/9/2/archivelist/year-2018,month-9,starttime-43345.cms',
 'https://timesofindia.indiatimes.com/2018/9/3/archivelist/year-2018,month-9,starttime-43346.cms',
 'https://timesofindia.indiatimes.com/2018/9/4/archivelist/year-2018,month-9,starttime-43347.cms',
 'https://timesofindia.indiatimes.com/2018/9/5/archivelist/year-2018,month-9,starttime-43348.cms',
 'https://timesofindia.indiatimes.com/2018/9/6/archivelist/year-2018,month-9,starttime-43349.cms',
 'https://timesofindia.indiatimes.com/2018/9/7/archivelist/year-2018,month-9,starttime-43350.cms',
 'https://timesofindia.indiatimes.com/2018/9/8/archivelist/year-2018,month-9,starttime-43351.cms',
 'https://timesofindia.indiatimes.com/2018/9/9/archivelist/year-2018,month-9,starttime-43352.cms',
 'https://timesofindia.indiatimes.com/2018/9/10/archivelist/year-2018,month-9,starttime-43353.cms',
 'https:/

In [18]:
len(dates)

30

In [34]:
################### traversing each date and collecting URL ###########################
news_urls=[]
cnt=1
for j in dates:
    browser.get(j)
    print("**************Collecting all news URL's on page ",cnt,"***********************")
    tble=browser.find_element_by_xpath("//table[@class='cnt'][2]")
    temp=tble.find_elements_by_xpath(".//tbody/tr[2]/td[1]/div[3]/table/tbody/tr[2]/td")
    for i in temp[0].find_elements_by_xpath(".//span/a"):
        news_urls.append(i.get_attribute('href'))
    for i in temp[2].find_elements_by_xpath(".//span/a"):
        news_urls.append(i.get_attribute('href'))
    print("************* All news URL's on page ",cnt, " have been stored successfully**********")
    cnt=cnt+1

**************Collecting all news URL's on page  1 ***********************
************* All news URL's on page  1  have been stored successfully**********
**************Collecting all news URL's on page  2 ***********************
************* All news URL's on page  2  have been stored successfully**********
**************Collecting all news URL's on page  3 ***********************
************* All news URL's on page  3  have been stored successfully**********
**************Collecting all news URL's on page  4 ***********************
************* All news URL's on page  4  have been stored successfully**********
**************Collecting all news URL's on page  5 ***********************
************* All news URL's on page  5  have been stored successfully**********
**************Collecting all news URL's on page  6 ***********************
************* All news URL's on page  6  have been stored successfully**********
**************Collecting all news URL's on page  7 *************

In [35]:
len(news_urls)

15000

In [62]:
def func(urls_):
    author=[]
    date=[]
    vertical=[]
    content=[]
    heading=[]
    ######################## traversing each URL one by one #########################
    for k in urls_:
        browser.get(k)
        #******************** auhtor ************************
        try:
            txt=browser.find_element_by_xpath("//div[@class='as_byline']/div[2]").text
            if(txt.find('IST')==-1): author.append(txt)
            else: 
                author.append('')
                #date.append(txt.replace('Updated:','').replace('Created:',''))
        except: 
            try:author.append(browser.find_element_by_xpath("//div[@class='_3Mkg- byline']/span").text)
            except:author.append('')
            
        #********************** dates **************************
        try:
            dt=browser.find_element_by_xpath("//div[@class='as_byline']/div[3]").text.replace('Updated:','').replace('Created:','')
            if(dt.find('IST')):date.append(dt)
            else: date.append('')
        except: 
            try:
                dt=browser.find_element_by_xpath("//div[@class='as_byline']/div[2]").text
                if(txt.find('IST')!=-1):date.append(dt.replace('Updated:','').replace('Created:',''))
                else:date.append('')  
            except:
                try:date.append(browser.find_element_by_xpath("//div[@class='_3Mkg- byline']").text[-23:])
                except:date.append('')
                
        #**************************** vertical ********************      
        try: vertical.append(browser.find_element_by_xpath("//div[@class='navbdcrumb']/div[@class='wrapper']/ol/li[2]").text)
        except: vertical.append('')
            
        #*************************** content ***********************
        try: content.append(browser.find_element_by_xpath("//div[@class='article_content clearfix']/arttextxml/div[@class='section1']/div[@class='Normal']").text.replace('\n',''))
        except: 
            try:content.append(browser.find_element_by_xpath("//div[@class='ga-headlines']").text.replace('\n',''))
            except:content.append('')
                
        #***************************** heading *********************
        try: heading.append(browser.find_element_by_xpath("//div[@class='as_heading']/h1[@class='heading1']").text)
        except: 
            try:heading.append(browser.find_element_by_xpath("//div[@class='_2NFXP ']/h1[@class='_23498']").text)
            except:heading.append('')
    df=pd.DataFrame({'Date':date,'Author':author,'Vertical':vertical,'Headline':heading,'Description':content})
    return df

In [69]:
df=func(news_urls)
df.to_csv('TOI_archive_data_sep2018.csv')